# Data pipelines involves extracting, transforming, & loading data for analytical use case

In [ ]:
! python ../setup.py

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

In [ ]:
%%sql
show tables;

## Let's create our data pipeline with Python

In [ ]:
import duckdb

db_file_name = 'tpch.db'
conn = duckdb.connect(db_file_name)
cursor = conn.cursor()

# Connect to DuckDB and load TPC-H tables into Pandas DataFrames
customer_df = cursor.sql("SELECT * FROM customer").df()
orders_df = cursor.sql("SELECT * FROM orders").df()
lineitem_df = cursor.sql("SELECT * FROM lineitem").df()
nation_df = cursor.sql("SELECT * FROM nation").df()
region_df = cursor.sql("SELECT * FROM region").df()
supplier_df = cursor.sql("SELECT * FROM supplier").df()
part_df = cursor.sql("SELECT * FROM part").df()
partsupp_df = cursor.sql("SELECT * FROM partsupp").df()

conn.close()

In [ ]:
# Create bronze tables
def create_bronze_tables(db_file_name):
    with duckdb.connect(db_file_name) as con:
        con.sql("""
        DROP SCHEMA IF EXISTS bronze CASCADE;
        """)
        
        con.sql("""
        CREATE SCHEMA IF NOT EXISTS bronze;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS bronze.customer;
        """)
        
        con.sql("""
        CREATE TABLE bronze.customer AS 
        SELECT 
            c_custkey AS customer_key,
            c_name AS name,
            c_address AS address,
            c_nationkey AS nationkey,
            c_phone AS phone,
            c_acctbal AS acctbal,
            c_mktsegment AS mktsegment,
            c_comment AS comment
        FROM customer;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS bronze.nation;
        """)
        
        con.sql("""
        CREATE TABLE bronze.nation AS 
        SELECT 
            n_nationkey AS nationkey,
            n_name AS name,
            n_regionkey AS regionkey,
            n_comment AS comment
        FROM nation;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS bronze.region;
        """)
        
        con.sql("""
        CREATE TABLE bronze.region AS 
        SELECT 
            r_regionkey AS regionkey,
            r_name AS name,
            r_comment AS comment
        FROM region;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS bronze.orders;
        """)
        
        con.sql("""
        CREATE TABLE bronze.orders AS 
        SELECT 
            o_orderkey AS orderkey,
            o_custkey AS custkey,
            o_orderstatus AS orderstatus,
            o_totalprice AS totalprice,
            o_orderdate AS orderdate,
            o_orderpriority AS orderpriority,
            o_clerk AS clerk,
            o_shippriority AS shippriority,
            o_comment AS comment
        FROM orders;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS bronze.lineitem;
        """)
        
        con.sql("""
        CREATE TABLE bronze.lineitem AS 
        SELECT 
            l_orderkey AS orderkey,
            l_partkey AS partkey,
            l_suppkey AS suppkey,
            l_linenumber AS linenumber,
            l_quantity AS quantity,
            l_extendedprice AS extendedprice,
            l_discount AS discount,
            l_tax AS tax,
            l_returnflag AS returnflag,
            l_linestatus AS linestatus,
            l_shipdate AS shipdate,
            l_commitdate AS commitdate,
            l_receiptdate AS receiptdate,
            l_shipinstruct AS shipinstruct,
            l_shipmode AS shipmode,
            l_comment AS comment
        FROM lineitem;
        """)

In [ ]:
# Create silver tables
def create_silver_tables(db_file_name):
    with duckdb.connect(db_file_name) as con:
        con.sql("""
        DROP TABLE IF EXISTS dim_customer;
        """)
        
        con.sql("""
        CREATE TABLE dim_customer AS
        SELECT 
            c.customer_key,
            c.name AS customer_name,
            c.address,
            c.phone,
            c.acctbal,
            c.mktsegment,
            n.name AS nation_name,
            n.comment AS nation_comment,
            r.name AS region_name,
            r.comment AS region_comment
        FROM bronze.customer AS c
        LEFT JOIN bronze.nation AS n ON c.nationkey = n.nationkey
        LEFT JOIN bronze.region AS r ON n.regionkey = r.regionkey;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS fct_orders;
        """)
        
        con.sql("""
        CREATE TABLE fct_orders AS
        SELECT 
            o.orderkey,
            o.custkey,
            o.orderstatus,
            o.totalprice,
            o.orderdate,
            o.orderpriority,
            o.clerk,
            o.shippriority,
            o.comment
        FROM bronze.orders AS o;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS fct_lineitem;
        """)
        
        con.sql("""
        CREATE TABLE fct_lineitem AS
        SELECT 
            l.orderkey,
            l.partkey,
            l.suppkey,
            l.linenumber,
            l.quantity,
            l.extendedprice,
            l.discount,
            l.tax,
            l.returnflag,
            l.linestatus,
            l.shipdate,
            l.commitdate,
            l.receiptdate,
            l.shipinstruct,
            l.shipmode,
            l.comment
        FROM bronze.lineitem AS l;
        """)
        con.commit()

In [ ]:
# Create gold tables
def create_obts(db_file_name):
    with duckdb.connect(db_file_name) as con:
        con.sql("""
        DROP TABLE IF EXISTS wide_orders;
        """)
        
        con.sql("""
        CREATE TABLE wide_orders AS
        SELECT o.*,
            c.*
        FROM fct_orders o
        LEFT JOIN dim_customer c 
        ON o.custkey = c.customer_key;
        """)
        
        con.sql("""
        DROP TABLE IF EXISTS wide_lineitem;
        """)
        
        con.sql("""
        CREATE TABLE wide_lineitem AS
        SELECT * FROM fct_lineitem;
        """)


def create_preagg_tables(db_file_name):
    with duckdb.connect(db_file_name) as con:
        con.sql("""
        DROP TABLE IF EXISTS order_lineitem_metrics;
        """)
        
        con.sql("""
        CREATE TABLE order_lineitem_metrics AS
        SELECT 
            orderkey AS order_key,
            COUNT(linenumber) AS num_lineitems
        FROM wide_lineitem
        GROUP BY orderkey;
        """)
        con.sql("""
        DROP TABLE IF EXISTS customer_outreach_metrics;
        """)
        
        con.sql("""
        CREATE TABLE customer_outreach_metrics AS
        SELECT 
            o.customer_key,
            o.customer_name,
            MIN(o.totalprice) AS min_order_value,
            MAX(o.totalprice) AS max_order_value,
            AVG(o.totalprice) AS avg_order_value,
            AVG(m.num_lineitems) AS avg_num_items_per_order
        FROM wide_orders AS o
        LEFT JOIN (
            SELECT 
                orderkey AS order_key,
                COUNT(linenumber) AS num_lineitems
            FROM wide_lineitem
            GROUP BY orderkey
        ) AS m ON o.orderkey = m.order_key
        GROUP BY o.customer_key, o.customer_name;
        """)

def create_gold_tables(db_file_name):
    create_obts(db_file_name)
    create_preagg_tables(db_file_name)

In [ ]:
def run_pipeline(db_file_name):
    print("==========CREATING BRONZE TABLES===================")
    create_bronze_tables(db_file_name)
    print("==========CREATING SILVER TABLES===================")
    create_silver_tables(db_file_name)
    print("==========CREATING GOLD TABLES===================")
    create_gold_tables(db_file_name)

In [ ]:
# Run the data pipeline 
db_file_name = './tpch.db'
run_pipeline(db_file_name)

In [ ]:
with duckdb.connect(db_file_name) as con:
    con.table("customer_outreach_metrics").show(max_rows=5)

Cons

* Code
* Bugs and maintanance
* adding new tables is tough
* need to split out table functions for modularity,

Just running SQL code, there is an easier way to do this with data build tool (dbt)

## dbt (data build tool) enables one to build data pipelines with SWE best practices

### dbt is a cli 

- One SQL select query (with any transformations you want) per file
- file name = model name (model can be SQL table/view/matreilaized view, etc)
- Data quality checks
- define documentation with yml file
- automatic data lineage diagram with a static webpage
- support for SCD2 creation

### The popularity of dbt is its ability to create data pipelines with SQL scripts

In [ ]:
! dbt --version

In [ ]:
! dbt init tpch_warehouse

Create a `profiles.yml` in the `tpch_warehouse` folder with the following content (note this would be already created for you)

```yml
---
config:
  send_anonymous_usage_stats: false
tpch_warehouse:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: ./dbt.duckdb
    prod:
      type: duckdb
      path: ./dbt-prod.duckdb
```

In [ ]:
! cd tpch_warehouse/ && python ../../setup.py --db_file dbt.duckdb

In [ ]:
! mkdir tpch_warehouse/raw_data

In [ ]:
import duckdb

db_file_name = './tpch_warehouse/dbt.duckdb'
conn = duckdb.connect(db_file_name)
cursor = conn.cursor()

for table in ['customer', 'orders', 'lineitem', 'nation', 'region', 'supplier', 'part', 'partsupp']:
    cursor.execute(f"COPY {table} TO './tpch_warehouse/raw_data/{table}.csv' (HEADER, DELIMITER ',');")

conn.commit()
conn.close()

In [1]:
! cd tpch_warehouse && dbt debug # Command to check connection to our warehouse

15:02:06  Running with dbt=1.8.8
15:02:06  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
15:02:06  dbt version: 1.8.8
15:02:06  python version: 3.12.4
15:02:06  python path: /home/josephkevinmachado/code/de_101/env/bin/python
15:02:06  os info: Linux-6.9.3-76060903-generic-x86_64-with-glibc2.35
15:02:07  Using profiles dir at /home/josephkevinmachado/code/de_101/4-Data-Pipeline/tpch_warehouse
15:02:07  Using profiles.yml file at /home/josephkevinmachado/code/de_101/4-Data-Pipeline/tpch_warehouse/profiles.yml
15:02:07  Using dbt_project.yml file at /home/josephkevinmachado/code/de_101/4-Data-Pipeline/tpch_warehouse/dbt_project.yml
15:02:07  adapter type: duckdb
15:02:07  adapter version: 1.9.0
15:02:07  Configuration:
15:02:07    profiles.yml file [OK found and valid]
15:02:07    dbt_project.yml file [OK found and valid]
15:02:07  Required dependencies:
15:02:07   - git [OK found]

15:02:07  

In [2]:
! rm -rf ./tpch_warehouse/models/*/.ipynb_checkpoints # always run before dbt run, caused by notebooks, no need to do this if performed via terminal

In [3]:
! cd tpch_warehouse/ && dbt run

15:02:09  Running with dbt=1.8.8
15:02:09  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
15:02:10  Registered adapter: duckdb=1.9.0
15:02:10  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.tpch_warehouse.example
15:02:10  Found 12 models, 10 data tests, 8 sources, 416 macros
15:02:10  
15:02:10  Concurrency: 1 threads (target='dev')
15:02:10  
15:02:10  1 of 12 START sql view model main.stg_customers ................................ [RUN]
15:02:11  1 of 12 OK created sql view model main.stg_customers ........................... [OK in 0.15s]
15:02:11  2 of 12 START sql view model main.stg_lineitems ................................ [RUN]
15:02:11  2 of 12 OK created sql view model main.stg_lineitems ........................... [OK in 0.11s]
15:02:11  3 of 12 START sql view model main.stg_n

In [4]:
! rm -rf ./tpch_warehouse/models/*/.ipynb_checkpoints # always run before dbt test, caused by notebooks, no need to do this if performed via terminal

zsh:1: no matches found: ./tpch_warehouse/models/*/.ipynb_checkpoints


In [5]:
! cd tpch_warehouse/ && dbt test

15:02:14  Running with dbt=1.8.8
15:02:14  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
15:02:15  Registered adapter: duckdb=1.9.0
15:02:15  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.tpch_warehouse.example
15:02:15  Found 12 models, 10 data tests, 8 sources, 416 macros
15:02:15  
15:02:15  Concurrency: 1 threads (target='dev')
15:02:15  
15:02:15  1 of 10 START test not_null_fct_orders_orderkey ................................ [RUN]
15:02:15  1 of 10 PASS not_null_fct_orders_orderkey ...................................... [PASS in 0.09s]
15:02:15  2 of 10 START test not_null_stg_nation_nationkey ............................... [RUN]
15:02:15  2 of 10 PASS not_null_stg_nation_nationkey ..................................... [PASS in 0.02s]
15:02:15  3 of 10 START test not_null_stg_ord

In [7]:
import duckdb

db_file_name = 'tpch.db'
conn = duckdb.connect(db_file_name)
cursor = conn.cursor()

# Connect to DuckDB and load TPC-H tables into Pandas DataFrames
customer_df = cursor.sql("SELECT * FROM dim_customer").df()

conn.commit()
conn.close()

In [12]:
customer_df.sample()

,customer_key,customer_name,address,phone,acctbal,mktsegment,nation_name,nation_comment,region_name,region_comment
695,696,Customer#000000696,"O2tHWRsd,mnrkAM2uw",32-923-916-5643,3778.97,HOUSEHOLD,RUSSIA,uctions. furiously unusual instructions sleep ...,EUROPE,e dolphins are furiously about the carefully


### dbt has a preferred step-by-step way to transform data

### Connection credentials are stored in profiles.yml

### Configuration settings are stored in dbt_project.yml

### Ensure the data is correct with tests


### Create SCD2 easily with dbt snapshot